In [1]:
import numpy as np
import pandas as pd

import os
import glob
import librosa
import tqdm
import pickle

from collections import Counter, defaultdict

from torchvision.datasets.folder import default_loader
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch.nn.functional as F
import torch

DATA_PATH = './data'
join_path = lambda path: os.path.join(DATA_PATH, path)

In [2]:
filepaths = glob.glob(join_path('train_curated/*')) # именно в таком порядке будут идти все аудиозаписи

train = pd.read_csv(join_path('train_curated.csv')).set_index('fname')

cats = list(set(sum(list(train['labels'].str.split(',').values), [])))

train['enc_labels'] = train['labels'].str.split(',').apply(lambda x: [int(label in x) for label in cats])

targets = np.array(train.loc[[fp[-12:] for fp in filepaths], 'enc_labels'].tolist())

In [31]:
import utils
import models
import trainer
import mclnn
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:


class conf:
    # Preprocessing settings
    sampling_rate = 44100
    duration = 2
    hop_length = 347*duration # to make time steps 128
#     hop_length = 694*duration
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    samples = sampling_rate * duration
    trim_long_data = False
    
conf.columns, conf.column_encoder = utils.read_columns()

train_x, train_y, _ = utils.read_dataset_split("./data/train_curated.csv", "./splits/split/train", conf, lambda s: s)
test_x, test_y, _ = utils.read_dataset_split("./data/train_curated.csv", "./splits/split/val", conf, lambda s: s)

all_rooms_meta, all_rooms = utils.read_filters("./filters/", conf, lambda s: s)

test_x = list(map(lambda s: utils.trim_and_mel(conf, s), test_x))

train_dataset = trainer.SoundAugDataset(train_x, train_y, 
                                        transform = lambda s: utils.audio_to_melspectrogram(conf, s),
                                        room_filters=all_rooms[:3], 
                                        params={
                                                    "max_size":len(train_x),
                                                    "max_n_mixed":2,
#                                                     "change_pitch":0.1,
#                                                     "change_pitch_max":1,
                                                    "add_echo":0.5,
#                                                     "harmonic_percussive":0.1,
#                                                     "noise_magnitude":0.00005
                                               },
                                        config=conf,
                                        seed=42)
test_dataset = trainer.SoundDataset(test_x, test_y)




100%|██████████| 73/73 [00:08<00:00,  8.12it/s]


In [33]:
aug_coll = trainer.AugmentationCollator(conf, None, True)
train_dl = DataLoader(train_dataset, batch_size=64, collate_fn=aug_coll, shuffle=True, num_workers=4)

val_dl = DataLoader(test_dataset, batch_size=64, collate_fn=trainer.collate_fn_2, shuffle=True)

In [36]:
import mclnn
base = nn.Sequential(
    mclnn.CLNNModule(128, 256, 4, dilation=1),
    nn.LeakyReLU(),
    mclnn.BatchNorm(256),
    mclnn.CLNNModule(256, 256, 4, dilation=1),
    nn.LeakyReLU(),
    mclnn.TemporalPool(2),
    mclnn.BatchNorm(256),
    mclnn.CLNNModule(256, 256, 4, dilation=1),
    nn.LeakyReLU(),
    mclnn.TemporalPool(2),
    mclnn.CLNNModule(256, 256, 4),
    nn.LeakyReLU(),
    mclnn.TemporalPool(4),
    mclnn.BatchNorm(256),
    mclnn.Flatten(),
    nn.Linear(256*8, 1024),
    nn.LeakyReLU(),
    nn.Linear(1024, 80)
)

class W(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

        
    def forward(self, input):
        input = input.squeeze(1)
        return self.model(input)
    
model = W(base)

In [37]:
# pip install scikit-multilearn 
# from skmultilearn.model_selection import iterative_train_test_split

# train_X, train_y, test_X, test_y = iterative_train_test_split(X=data, y=targets, test_size=0.1)

In [38]:
# train_ds = TensorDataset(torch.tensor(train_X).unsqueeze(1), torch.tensor(train_y).float())
# val_ds = TensorDataset(torch.tensor(test_X).unsqueeze(1), torch.tensor(test_y).float())

# train_dl = DataLoader(train_ds, batch_size=64)
# val_dl = DataLoader(val_ds, batch_size=64)

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = Model(num_classes=80)
logdir = join_path('logdir')
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [40]:
from kirill.flame import Trainer # мой небольшой пайплайн для тренировки сеток

trainer = Trainer(model, optimizer, criterion, device, logdir)

CRITICAL:root:Cannot load caffe2.python. Error: /opt/caffe2/build/caffe2/python/caffe2_pybind11_state.so: undefined symbol: _ZN5fLI6438FLAGS_caffe2_max_keep_on_shrink_memoryE


In [41]:
# # пример запуска для минимизации логлосса
trainer.train(train_dl, val_dl, n_epochs=240, verbose=True)


# losses, best_loss, epoch_time = utils.train_model(trainer, train_dl, val_dl, val_y, scheduler,
#                                             n_epochs=1000, gap=20, verbose=True)



  1 epoch. train: 0.00198. val: 0.00121. time: 76.167s.
  2 epoch. train: 0.00159. val: 0.00121. time: 151.688s.
  3 epoch. train: 0.00149. val: 0.00121. time: 227.603s.
  4 epoch. train: 0.00137. val: 0.00124. time: 302.526s.
  5 epoch. train: 0.00124. val: 0.00125. time: 379.171s.
  6 epoch. train: 0.00109. val: 0.00138. time: 455.731s.
  7 epoch. train: 0.00091. val: 0.00147. time: 531.790s.
  8 epoch. train: 0.00076. val: 0.00176. time: 607.959s.
  9 epoch. train: 0.00068. val: 0.00171. time: 684.863s.
 10 epoch. train: 0.00065. val: 0.00210. time: 760.980s.
 11 epoch. train: 0.00062. val: 0.00243. time: 838.014s.
 12 epoch. train: 0.00061. val: 0.00236. time: 914.303s.
 13 epoch. train: 0.00062. val: 0.00241. time: 993.766s.
 14 epoch. train: 0.00056. val: 0.00261. time: 1074.840s.
 15 epoch. train: 0.00045. val: 0.00267. time: 1155.898s.
 16 epoch. train: 0.00037. val: 0.00261. time: 1235.745s.
 17 epoch. train: 0.00029. val: 0.00253. time: 1315.813s.
 18 epoch. train: 0.00023. v

OSError: [Errno 12] Cannot allocate memory

In [18]:
from metrics import lwlrap_score

# если хочется считать скор метрики оргов
losses = []
for i in range(20):
    # делает один проход по даталоадеру
    trainer.train_step(train_dl, verbose=True)
    
    # формируем предсказания
    model.eval()
    preds = []
    for batch in tqdm.tqdm(val_dl):
        batch = batch[0].to(device)
        pred = model(batch)
        preds.append(pred.cpu().detach().numpy())
    preds = np.vstack(preds)
    
    # считаем скор метрики оргов
    score = lwlrap_score(test_y, preds)
    print(score)
    
    losses.append(score)

  0%|          | 0/70 [00:00<?, ?it/s]

0.32207662069576626


  0%|          | 0/70 [00:00<?, ?it/s]

0.4166380930591757


  0%|          | 0/70 [00:00<?, ?it/s]

...

0.6772803548081768


  0%|          | 0/70 [00:00<?, ?it/s]

0.6778412922729062


100%|██████████| 8/8 [00:00<00:00, 11.81it/s]

0.6585010786758971


In [ ]:
# test inference

with open(join_path('test_mels.pkl'), 'rb') as f:
    test_data = pickle.load(f)
    
test_filepaths = glob.glob(join_path('test/*'))

test_ds = TensorDataset(torch.tensor(test_data).unsqueeze(1))
test_dl = DataLoader(test_ds, batch_size=64)

model.eval()
preds = []
for batch in tqdm.tqdm(test_dl):
    batch = batch[0].to(device)
    pred = model(batch)
    preds.append(pred.cpu().detach().numpy())
preds = np.vstack(preds)

In [ ]:
sub = pd.DataFrame(preds)
sub.columns = [label for label in cats]
sub['fname'] = [fp[-12:] for fp in test_filepaths]
sub.to_csv("sub3.csv", index=False)